**In this notebook, we will set up the R square for treeshap. Before that, we will first re-implement treeSHAP using the new framework through complex root of unity**

# Tree Summary
Before we start the new algorithm, we introduce some code to retrieve the informationto be used from the tree of other modules

In [2]:
import numpy as np
import scipy
import shap
from sklearn.tree import DecisionTreeRegressor
import sklearn.ensemble
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import time as time
from numba import njit
from sklearn.datasets import make_regression
from dataclasses import dataclass
import numpy as np
from dataclasses import dataclass


@dataclass(frozen=True)
class simple_tree:
    """
    dataclass for a simple tree, in the scikit learn format that is necessary for computation
    
    Data:
    children_left: left children_index
    children_right: right children_index
    feature: array of features splitted at each node
    threshold: array of threshols for corresponding splitting features
    max_depth: max_depth of the tree
    n_node_samples: array of sample size for each node
    value: array of values for each node, only leaf value is used, so only keep leaf value is fine
    node_count: total number of leaves 
    """
    children_left: np.ndarray
    children_right: np.ndarray
    feature: np.ndarray
    threshold: np.ndarray
    max_depth: int
    n_node_samples: np.ndarray
    value: np.ndarray
    node_count: int


@dataclass(frozen=True)
class tree_summary:
    """
    dataclass for the calculation of cd-treeshap family
    
    Data:
    - children_left: left children_index
    - children_right: right children_index 
    - feature: array of features splitted at each node
    - feature_uniq: array of uniq features
    - threshold: array of threshols for corresponding splitting features
    - max_depth: the max_depth of the tree
    - sample_weight: list of sample size of parent/sample size of current node
    - init_prediction: initial prediction from each leaf
    - node_count: number of nodes
    """
    children_left: np.ndarray
    children_right: np.ndarray
    feature: np.ndarray
    feature_uniq: np.ndarray
    threshold: np.ndarray
    max_depth: int
    sample_weight: np.ndarray
    init_prediction: np.ndarray
    node_count: int
    

    
def summarize_tree(tree):
    """
    Summarize the data needed for tree_summary. The tree object should have:
    children_left: left children_index
    children_right: right children_index
    feature: array of features splitted at each node
    threshold: array of threshols for corresponding splitting features
    max_depth: max_depth of the tree
    n_node_samples: array of sample size for each node
    value: array of values for each node, only leaf value is used, so only keep leaf value is fine
    node_count: total number of leaves 
    """
    sample_weight = np.ones_like(tree.threshold)
    init_prediction = np.zeros_like(tree.threshold)
    n = tree.n_node_samples[0]
    
    def traversal_summarize_tree(v):
        v_l, v_r = tree.children_left[v], tree.children_right[v]
        n_v, n_l, n_r = tree.n_node_samples[v], tree.n_node_samples[v_l], tree.n_node_samples[v_r]
        
        if v_l < 0:  #leaf
            init_prediction[v] = tree.value[v] * n_v/n
        else:
            sample_weight[v_l], sample_weight[v_r] = n_v/n_l, n_v/n_r
            traversal_summarize_tree(v_l)
            traversal_summarize_tree(v_r)
    
    # travel from the root
    traversal_summarize_tree(0)
    
    feature_uniq = np.unique(tree.feature[tree.feature >= 0])
   
    return tree_summary(tree.children_left, tree.children_right, tree.feature, feature_uniq, tree.threshold, tree.max_depth, sample_weight, init_prediction, tree.node_count)



# Begin testing
# Generate synthetic data
np.random.seed(0)
n_samples = 2000
max_depth = 6
X = np.random.rand(n_samples, 3) 
y = X[:, 0] + 2 * X[:, 1] + 0.5 * X[:, 2] + np.random.randn(n_samples)  # Dependent variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a Decision Tree Regressor
tree_regressor = DecisionTreeRegressor(max_depth = max_depth)  # You can adjust max_depth as needed


# Fit the Decision Tree to the training data
tree_fit = tree_regressor.fit(X_train, y_train)
start = time.time()
summary_tree = summarize_tree(tree_fit.tree_)
end = time.time()
print(end - start)
print(summary_tree)        

if max_depth < 5:  
    from sklearn.tree import plot_tree
    plt.figure(figsize=(12, 6))
   # plot_tree(tree_regressor, filled=True, feature_names=['Feature 1', 'Feature 2', 'Feature 3'])
    plot_tree(tree_regressor, filled=True)
    plt.show()

0.0005939006805419922
tree_summary(children_left=array([  1,   2,   3,   4,   5,   6,  -1,  -1,   9,  -1,  -1,  12,  13,
        -1,  -1,  16,  -1,  -1,  19,  20,  21,  -1,  -1,  -1,  25,  26,
        -1,  -1,  29,  -1,  -1,  32,  33,  34,  35,  -1,  -1,  38,  -1,
        -1,  41,  42,  -1,  -1,  45,  -1,  -1,  48,  49,  50,  -1,  -1,
        53,  -1,  -1,  56,  57,  -1,  -1,  -1,  61,  62,  63,  64,  65,
        -1,  -1,  68,  -1,  -1,  71,  72,  -1,  -1,  75,  -1,  -1,  78,
        79,  80,  -1,  -1,  -1,  84,  85,  -1,  -1,  88,  -1,  -1,  91,
        92,  93,  94,  -1,  -1,  97,  -1,  -1, 100, 101,  -1,  -1, 104,
        -1,  -1, 107, 108, 109,  -1,  -1, 112,  -1,  -1, 115, 116,  -1,
        -1, 119,  -1,  -1], dtype=int64), children_right=array([ 60,  31,  18,  11,   8,   7,  -1,  -1,  10,  -1,  -1,  15,  14,
        -1,  -1,  17,  -1,  -1,  24,  23,  22,  -1,  -1,  -1,  28,  27,
        -1,  -1,  30,  -1,  -1,  47,  40,  37,  36,  -1,  -1,  39,  -1,
        -1,  44,  43,  -1,  -1

/var/folders/rn/qq25rs5s3wn9yqrktd830cvc0000gp/T/ipykernel_12542/2204973113.py:90: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  init_prediction[v] = tree.value[v] * n_v/n


# Miscellaneous functions
This module will introduce some functions used for calculating treeshap

In [3]:
d_test = 9

def inv_binom_coef(d):
    """
    calculate the inverse of binomial coefficients using an iterative method with symmetry.
    
    Parameters:
    - d: dimension
    
    Example: 
    binom_coef(5)
    ([ 1.,  5., 10., 10.,  5.,  1.])
    """
    coef = np.zeros(d + 1)
    coef[0] = 1
    for i in range(1, d // 2 + 1):
        coef[i] = coef[i - 1] * (d - i + 1) / i
    for i in range(d // 2 + 1, d + 1):
        coef[i] = coef[d - i]
    return 1/coef

#inv_binom_coef(d_test)

def complex_v_invc_degree(d):
    """
    Pre store v_invc: v(z)^-1 @ c / d at degree d where z are complex roots of unity
    
    Parameters:
    -d: degree
    -c: the coefficients matrix
    """
    omega_inv = np.exp(-2 * np.pi * 1j * np.arange(d) / d)
    v_omega_inv = np.vander(omega_inv, increasing=True)
    v_inv_omega_theo = v_omega_inv / d
    res = v_inv_omega_theo @ inv_binom_coef(d-1) / d
    return res

print(complex_v_invc_degree(d_test))


def store_complex_v_invc(d):
    """
    Pre store v_invc: v(z)^-1 @ c / d up to maximum tree depth where z are complex roots of unity
    
    Parameters:
    -d: max treedepth
    """
    res = np.zeros((d+1, d), dtype=complex)
    
    for i in range(1, d+1):
        res[i, :i] = complex_v_invc_degree(i)
    
    return res 

# how to retrive the degree d_test pre_stored value
print(store_complex_v_invc(20)[d_test, :d_test])

def store_complex_root(d):
    """
    Prestore the complex root of unity z
    
    Parameters:
    -d: max treedepth
    """ 
    res = np.zeros((d+1, d), dtype=complex)
    
    for i in range(1, d+1):
        res[i, :i] = np.exp(2 * np.pi * 1j * np.arange(i) / i)
    return res


@njit
def complex_dot_v2(p, v_invc, d):
    """
    Return the dot product: C(z) * P(z) / d where z are the complex roots of unity, using the fact that P(w) and v_invc both:
    except for the 0 index and possibly the last when d is odd, the rest are complex conjugate by head and tail, etc...
    
    Parameters 
    - p: a polynomial vector evaluated at complex root of unity
    - v_invc: pre-calculated inverse coefficients
    - d: the original degree before cut by half.
    """
    len_p = len(p)
    res = p[0] * v_invc[0]
    if d % 2 == 0:
        res += 2 * np.dot(p[1:(len_p-1)], v_invc[1:(len_p-1)]) + p[-1] * v_invc[-1]
    else:
        res += 2 * np.dot(p[1:len_p], v_invc[1:len_p])
    return res.real

# Example usage with numpy arrays
import numpy as np

v1 = np.array([3, 6-1j, 5+1j, 6+1j])
v2 = np.array([5, 4-1j, 6+1j, 4+1j])
v3 = np.array([3, 6-1j, 5+1j])
v4 = np.array([5, 4-1j, 6+1j])
print(complex_dot_v2(v3, v4, len(v1)))

v5 = np.array([3, 6-1j, 5+1j, 5-1j, 6+1j])
v6 = np.array([5, 4-1j, 6+1j, 6-1j, 4+1j])
v7 = np.array([3, 6-1j, 5+1j])
v8 = np.array([5, 4-1j, 6+1j])
print(complex_dot_v2(v7, v8, len(v5)))

[0.0292769 +0.j         0.02262614+0.00823524j 0.01286629+0.0107961j
 0.00487213+0.00843878j 0.00055254+0.00313362j 0.00055254-0.00313362j
 0.00487213-0.00843878j 0.01286629-0.0107961j  0.02262614-0.00823524j]
[0.0292769 +0.j         0.02262614+0.00823524j 0.01286629+0.0107961j
 0.00487213+0.00843878j 0.00055254+0.00313362j 0.00055254-0.00313362j
 0.00487213-0.00843878j 0.01286629-0.0107961j  0.02262614-0.00823524j]
90.0
119.0


# Iterative way

This weight matrix only calculates weight for leaves

In [4]:
@njit
def traversal_weight(x, v, w, children_left, children_right, feature, threshold, sample_weight, leaf_ind, w_res, w_ind, depth, met_feature):
    """
    Calculate the weight in the treeSHAP. 

    Parameters:
    - x: one sample to be explained 
    - v: node index 
    - w: weight vector passed to the current node, for temporary usage
    - children_left: left children_index
    - children_right: right children_index 
    - feature: list of features splitted at each node
    - threshold: list of threshold for corresponding features
    - sample_weight: a list of sample size of parent/sample size of current node
    - leaf_ind: leaf indices
    - w_res, L * p matrix of weights, which records the modified weight for each leaf and each feature
    - w_ind, L * p matrix of indicator matrix, which records the met of features for each leaf
    - depth: current depth
    - met_feature: record all the features met to now

    Update:
    w_res
    w_ind
    met_feature
    """

    v_l, v_r = children_left[v], children_right[v]

    if v_l < 0:
        # match to the right location so the value for w_res corresponds to the same order of leaf_ind
        ind = (leaf_ind == v)
        #feature_tmp = met_feature[0:depth]
        for tmp_depth in range(depth):
            w_res[ind, met_feature[tmp_depth]] = w[tmp_depth]
            w_ind[ind, met_feature[tmp_depth]] = 1
    else:
        split_feature = feature[v]
        split_threshold = threshold[v]

        former_depth = np.arange(depth)[met_feature[0:depth] == split_feature]

        if len(former_depth) != 0:
            former_depth = former_depth[-1]
        else:
            former_depth = depth  
            w[depth] = 1

        met_feature = met_feature.copy()
        met_feature[depth] = split_feature

        w_r = w.copy()

        if x[split_feature] <= split_threshold:
            w[depth] = w[former_depth] * sample_weight[v_l]
            w_r[depth] = 0
        else:
            w_r[depth] = w_r[former_depth] * sample_weight[v_r]
            w[depth] = 0 

        traversal_weight(x, v_l, w, children_left, children_right, feature, threshold, sample_weight, leaf_ind, w_res, w_ind, depth+1, met_feature) 
        traversal_weight(x, v_r, w_r, children_left, children_right, feature, threshold, sample_weight, leaf_ind, w_res, w_ind, depth+1, met_feature)

        
def weight(x, summary_tree):
    p = len(x)
    d = summary_tree.max_depth
    
    feature_uniq = summary_tree.feature_uniq
    
    leaf_ind = np.arange(summary_tree.node_count)[summary_tree.children_left==-1]

    # L * p matrix. Note that unused features are also labeled as 0 here for efficient storage.
    w_res = np.empty((len(leaf_ind), p))
    w_res[:, feature_uniq] = 1
    
    w = np.empty(d)
    
    # L * p matrix. [i, j] = 1 if the feature j is used by leaf i. [i, j] = 0 corresponds to 1 in the
    # above matrix. This two sparse matrices together could make the weight matrix well-defined, and save the
    # storage at the same time
    w_ind = np.empty((len(leaf_ind), p))
    w_ind[:, feature_uniq] = 0
    
    met_feature = np.full(d, -1, dtype=int)
    
    # begin traversal from root
    traversal_weight(x, 0, w, summary_tree.children_left, summary_tree.children_right, summary_tree.feature, summary_tree.threshold, summary_tree.sample_weight, leaf_ind, w_res, w_ind, 0, met_feature)
    return w_res, w_ind


# test
ind = 0
x = X_test[ind]
print("First sample:" + str(x))
start = time.time()
a = weight(x, summary_tree)
end = time.time()
print("\n time: " + str(end - start) + "\n")
print(a[0])
print(a[1])


from line_profiler import LineProfiler

def main():
    # Assuming x and summary_tree are defined and set up correctly
    lp = LineProfiler()
    # Add the function to be profiled
    lp.add_function(weight)
    # Optionally, if you want to profile traversal_weight and it's called inside weight, add it too
    lp.add_function(traversal_weight.__wrapped__)
    
    # Run the function multiple times
    for _ in range(10000):  # Adjust the number of runs as needed
        lp.runcall(weight, x, summary_tree)
    
    # Print out the profiling results
    lp.print_stats()

if __name__ == "__main__":
    main()

# if __name__ == "__main__":
#     profiler = cProfile.Profile()
#     profiler.run('main()')
#     stats = pstats.Stats(profiler)
#     stats.strip_dirs().sort_stats('cumulative').print_stats(50)  
    


First sample:[0.06965074 0.80381453 0.18167326]

 time: 0.3003041744232178

[[3.         0.         1.97849462]
 [3.         0.         1.97849462]
 [4.64705882 0.         1.97849462]
 [0.         0.         1.97849462]
 [3.         0.         0.        ]
 [3.         0.         0.        ]
 [6.         0.         0.        ]
 [0.         0.         0.        ]
 [3.         0.         0.        ]
 [3.         0.         0.        ]
 [3.         0.         0.        ]
 [3.         0.         1.08      ]
 [3.         0.         1.08      ]
 [3.         0.         1.08      ]
 [3.         0.         1.08      ]
 [0.         0.         1.89285714]
 [0.         0.         1.89285714]
 [0.         0.         0.        ]
 [0.         0.         0.        ]
 [0.         0.         2.53153153]
 [0.         0.         0.        ]
 [0.         0.         0.        ]
 [0.         0.         0.        ]
 [0.         0.         3.        ]
 [0.         0.         0.        ]
 [0.         0.         

In [5]:
res = np.zeros_like(x)
for i in range(x.shape[0]):
    res[i] = x[i] * i

print(res)

[0.         0.80381453 0.36334652]


In [6]:
import cProfile
import concurrent.futures
import os
from numba import complex128, int32

@njit
def T2_sample(i, w_matrix, w_ind, init_prediction, store_v_invc, store_z, shap_value, feature_uniq):
    ## Calculate T2 for each sample
    L, p = w_matrix.shape
    
    for l1 in range(L):
        for l2 in range(l1, L):
            init_prediction_product = init_prediction[l1] * init_prediction[l2]
            
            union_f12 = feature_uniq[(w_ind[l1, feature_uniq] + w_ind[l2, feature_uniq]) >= 1]
            
            n12 = len(union_f12)
            # begin to use the property of complex conjugate
            n12_c = n12 // 2 + 1

            v_invc = store_v_invc[n12, :n12_c]
            z = store_z[n12, :n12_c]

            p_z = np.zeros((n12_c), dtype=complex128)
            tmp_p_z = np.zeros((n12_c), dtype=complex128)

            for k in range(n12_c):
                p_z[k] = np.prod(z[k] + w_matrix[l1, union_f12] * w_matrix[l2, union_f12])

            # update only when feature j belongs to the union of f1 and f2
            for j in union_f12:
                    # remove the operation for j by dividing
                for k in range(n12_c):
                    tmp_p_z[k] = p_z[k] / (z[k] + w_matrix[l1, j] * w_matrix[l2, j])
                    
                if l1 != l2:
                    shap_value[i, j] += 2 * (w_matrix[l1, j] * w_matrix[l2, j] - 1) * init_prediction_product * complex_dot_v2(tmp_p_z, v_invc, n12)
                else:
                    shap_value[i, j] += (w_matrix[l1, j] * w_matrix[l2, j] - 1) * init_prediction_product * complex_dot_v2(tmp_p_z, v_invc, n12)

                    
def T2(x, summary_tree, ncore=-1):
    """
    Calculate the second order treeshap value
    
    Parameters:
    -x: sample to be explained
    -summary_tree: summary tree
    -ncore: number of cores to use, with default value -1 to utilize all the cores
    
    Return:
    treeshap value for the sample
    """
    max_core = os.cpu_count()
    if ncore == -1:
         ncore = os.cpu_count()
    ncore = min(max_core, ncore)
    
    init_prediction = summary_tree.init_prediction[summary_tree.init_prediction!=0]
    d = summary_tree.max_depth
    
    # store v_inc * c / d evaluated at complex roots 
    store_v_invc = store_complex_v_invc(d * 2)
    store_z = store_complex_root(d * 2)
    
    shap_value = np.zeros_like(x)
    
    for i in range(x.shape[0]):
            xi = x[i]
            w = weight(xi, summary_tree)
            w_matrix = w[0]
            w_ind = w[1]

            T2_sample(i, w_matrix, w_ind, init_prediction, store_v_invc, store_z, shap_value, summary_tree.feature_uniq)

    return shap_value


start = time.time()
# Re-run the first trunk 
ind = range(100)
x = np.array(X_test[ind])
y = np.array(y_test[ind])
# x.shape = 1, 3

print("Sample: " + str(x) + "\n")

T2_x = T2(x, summary_tree)
print(T2_x)
print("T2 time: " + str(time.time() - start))

#print("Sample: " + str(x) + "\n")

# def main():
#     for _ in range(10):
#         T2_x = T2(x, summary_tree)
#     #print(T2_x)
#     pass

# if __name__ == "__main__":
#     cProfile.run('main()', 'main.stats')
#     main()
    

# # Load the stats file
# p = pstats.Stats('main.stats')

# #Sort the statistics by cumulative time and print the first few lines
# p.sort_stats('cumulative').print_stats(100)


def main():
    # Assuming x and summary_tree are defined and set up correctly
    lp = LineProfiler()
    # Add the function to be profiled
    lp.add_function(T2)
    
    lp.runcall(T2, x, summary_tree)
    
    # Print out the profiling results
    lp.print_stats()

if __name__ == "__main__":
    main()

Sample: [[0.06965074 0.80381453 0.18167326]
 [0.55805124 0.70494806 0.41863686]
 [0.30833843 0.29264205 0.56651827]
 [0.24499021 0.59294632 0.61242525]
 [0.54082552 0.75825584 0.15775159]
 [0.39113571 0.05202981 0.70457372]
 [0.70343042 0.33530341 0.10499074]
 [0.45956372 0.49306119 0.26039716]
 [0.49045881 0.22741463 0.25435648]
 [0.99337799 0.27436829 0.95224862]
 [0.72999056 0.17162968 0.52103661]
 [0.24804633 0.3315698  0.6317694 ]
 [0.10097628 0.42064702 0.73145025]
 [0.70340703 0.35307496 0.15442542]
 [0.96239507 0.11052229 0.63083181]
 [0.93239394 0.2153982  0.85833764]
 [0.12354668 0.46304438 0.91605091]
 [0.82076712 0.90884372 0.81552382]
 [0.04728931 0.28924748 0.73662394]
 [0.89996731 0.83447614 0.39913624]
 [0.33851449 0.57749619 0.85273616]
 [0.23542762 0.1528502  0.69291755]
 [0.10551829 0.28534037 0.3931698 ]
 [0.00924711 0.75249141 0.00623777]
 [0.10214669 0.54251256 0.60919391]
 [0.86219152 0.97291949 0.96083466]
 [0.1002582  0.22771359 0.72743971]
 [0.97021294 0.71748

# square_treeshap, loss_treeshap, and cd_treeshap

In [7]:
def loss_treeshap(x, y, summary_tree, explainer, learning_rate=1):
    """
    Explain l2 loss for every sample
    
    Parameters:
    -x: samples
    -y: y corresponding to x
    -summary_tree: summary tree
    -learning_rate: learning_rate if it's a tree from ensemble methods, learning_rate for decision treeshould be 1
    
    Return:
    Global treeshap value
    """
    square_treeshap_x = T2(x, summary_tree) * learning_rate ** 2 
    # direct call from shap
    T0_x = explainer.shap_values(x) * learning_rate 
    res = square_treeshap_x - 2 * (y * T0_x.T).T
    return res 

def cd_treeshap(x, y, summary_tree, explainer):
    """
    Calculate the cd-treeshap value
    
    Parameters:
    -x: samples
    -y: y corresponding to x
    -summary_tree: summary tree
    
    Return:
    Global treeshap value
    """
    sst = np.sum((y-np.mean(y)) ** 2)
    T0_x = explainer.shap_values(x)
    square_shap = T2(x, summary_tree)
    res_indv = 1/sst * (-square_shap + 2 * (y * T0_x.T).T)
    res = np.sum(res_indv, axis = 0)
    return res


# Re-run the first trunk 
ind = range(3)
x = np.array(X_test[ind])
y = np.array(y_test[ind])
ypred = tree_regressor.predict(x)
# x.shape = 1, 3

explainer = shap.TreeExplainer(tree_regressor)
square_treehshap_x = T2(x, summary_tree)
print("square_treeshp: " + str(square_treehshap_x) + "\n")


loss_treeshap_xy = loss_treeshap(x, y, summary_tree, explainer)
print("My loss treeshap: " + str(loss_treeshap_xy))
print("Loss sum + intercept: " + str(np.sum(loss_treeshap_xy, axis=1) + (y - np.mean(y_train)) ** 2))
print("Real loss: " + str((y - ypred) ** 2) + "\n")


x = np.array(X_train)
y = np.array(y_train)
# x.shape = 1, 3
cd_treeshap_res = cd_treeshap(x, y, summary_tree, explainer)
print("cd_treeshap: " + str(cd_treeshap_res))

# Let's check the real R^2
ypred = tree_regressor.predict(x)
sst = np.sum((y - np.mean(y)) ** 2)
sse = np.sum((y - ypred) ** 2)
rsq = 1 - sse/sst
print("The Model R^2 is: " + str(rsq) + "\n")
print("My sum of cd_treeshap and the real R^2 is equal?:  " + str(round(np.sum(cd_treeshap_res), 5)==round(rsq, 5)) + "!!!!!")

square_treeshp: [[-1.16487043  3.11592778 -0.60037881]
 [ 0.03772056  1.7066539  -0.553281  ]
 [-0.88915477 -1.49491777  0.62660414]]

My loss treeshap: [[-0.65075585  1.82708098 -0.35230783]
 [ 0.01748439  1.11573346 -0.35052859]
 [-0.40682569 -0.70104209  0.26213723]]
Loss sum + intercept: [1.81749189 1.97890908 0.15108101]
Real loss: [1.81749189 1.97890908 0.15108101]

cd_treeshap: [0.11493914 0.29050581 0.04686456]
The Model R^2 is: 0.45230950592279173

My sum of cd_treeshap and the real R^2 is equal?:  True!!!!!


In [8]:
isinstance(tree_regressor, sklearn.tree.DecisionTreeRegressor)

True

**This test is tiny, let's get a larger one, which could not be easily computed by moden computer by brute force**

In [9]:
import pstats

np.random.seed(0)
x, y, coefficients = make_regression(n_samples=1000, n_features=1000, n_informative=5, coef=True, random_state=0)

max_depth = 7
tree_regressor = DecisionTreeRegressor(max_depth=max_depth)
tree_fit = tree_regressor.fit(x, y)

summary_tree = summarize_tree(tree_fit.tree_)

start = time.time()
explainer = shap.TreeExplainer(tree_regressor)
cd_treeshap_res = cd_treeshap(x, y, summary_tree, explainer)
end = time.time()

print("time: " + str(end - start))
#print("cd_treeshap: " + str(cd_treeshap_res))

# Let's check the real R^2
ypred = tree_regressor.predict(x)
sst = np.sum((y - np.mean(y)) ** 2)
sse = np.sum((y - ypred) ** 2)
rsq = 1 - sse/sst

def main():
    cd_treeshap(x, y, summary_tree, explainer)
    pass

if __name__ == "__main__":
    cProfile.run('main()', 'main.stats')
    main()
    

# Load the stats file
p = pstats.Stats('main.stats')

# Sort the statistics by cumulative time and print the first few lines
p.sort_stats('cumulative').print_stats(100)

print(cd_treeshap_res)
print("My R^2 sum is: " + str(np.sum(cd_treeshap_res)))
print("Model R^2 is: " + str(rsq) + "\n")
print("My sum of cd_treeshap and the model R^2 is equal?:  " + str(round(np.sum(cd_treeshap_res), 5)==round(rsq, 5)) + "!!!!!")

/var/folders/rn/qq25rs5s3wn9yqrktd830cvc0000gp/T/ipykernel_12542/2204973113.py:90: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  init_prediction[v] = tree.value[v] * n_v/n


time: 11.992902755737305
Wed Feb  5 10:34:05 2025    main.stats

         11765 function calls (11761 primitive calls) in 11.734 seconds

   Ordered by: cumulative time
   List reduced from 193 to 100 due to restriction <100>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      2/1    0.000    0.000   11.729   11.729 {built-in method builtins.exec}
      2/1    0.191    0.096   11.729   11.729 <string>:1(<module>)
       10    9.144    0.914    9.291    0.929 {built-in method time.sleep}
        1    0.001    0.001    7.230    7.230 /var/folders/rn/qq25rs5s3wn9yqrktd830cvc0000gp/T/ipykernel_12542/2051348148.py:26(main)
        1    0.004    0.004    7.230    7.230 /var/folders/rn/qq25rs5s3wn9yqrktd830cvc0000gp/T/ipykernel_12542/1601285480.py:20(cd_treeshap)
        3    0.000    0.000    4.293    1.431 /opt/anaconda3/lib/python3.12/asyncio/base_events.py:1910(_run_once)
        3    0.369    0.123    4.281    1.427 /opt/anaconda3/lib/python3.12/selectors.py:55

# GBM

In [10]:
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.datasets import fetch_california_housing

# Load the California housing dataset
california_housing = fetch_california_housing()

# Extract features (X) and target variable (y)
X, y = california_housing.data, california_housing.target


# Train GBM model
gbm = GradientBoostingRegressor(n_estimators=10, max_depth=2)
gbm.fit(X, y)


rf = RandomForestRegressor(n_estimators=10)
rf.fit(X, y)

# Extract trees from the GBM model
trees = gbm.estimators_

# Summarize each tree
summary_trees = []
for tree in trees:
    summary_trees.append(summarize_tree(tree[0].tree_))


# Now you have summary_trees, which contains the summarized information of each tree
summary_trees

first_pred = trees[0, 0].predict(X)
second_pred = first_pred + trees[1, 0].predict(X)
third_pred = second_pred + trees[2, 0].predict(X)

final_pred = gbm.predict(X)

#print(first_pred)
#print(second_pred)
#print(y)

print(np.corrcoef(y, first_pred))
print(np.corrcoef(y, second_pred))
print(np.corrcoef(y, third_pred))
print(np.corrcoef(y, final_pred))

# get preidction for each stage  
staged_predictions = list(gbm.staged_predict(X))[0]
print(staged_predictions)
print(y)

len(trees)

explainer = shap.TreeExplainer(trees[1, 0])
print(explainer)

[[1.         0.66874098]
 [0.66874098 1.        ]]
[[1.        0.6817336]
 [0.6817336 1.       ]]
[[1.         0.70735073]
 [0.70735073 1.        ]]
[[1.         0.74599082]
 [0.74599082 1.        ]]
[2.28334546 2.28334546 2.28334546 ... 1.99739531 1.99739531 1.99739531]
[4.526 3.585 3.521 ... 0.923 0.847 0.894]


/var/folders/rn/qq25rs5s3wn9yqrktd830cvc0000gp/T/ipykernel_12542/2204973113.py:90: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  init_prediction[v] = tree.value[v] * n_v/n


In [11]:
isinstance(gbm, sklearn.ensemble.GradientBoostingRegressor)
gbm.max_depth

2

In [14]:
x, y, coefficients = make_regression(n_samples=2000, n_features=100, n_informative=10, coef=True, random_state=0)

tree_regressor = GradientBoostingRegressor(max_depth=3)
#tree_regressor = RandomForestRegressor(n_estimators=100, max_depth=max_depth, learning_rate = 0.01)

tree_fit = tree_regressor.fit(x, y)
ypred = tree_regressor.predict(x)
ensemble_tree = tree_regressor.estimators_
staged_predict = list(tree_fit.staged_predict(x))
#learning rate
alpha = tree_regressor.learning_rate

num_tree = len(tree_fit)

start = time.time()
loss = np.zeros_like(x)
for i in range(num_tree):
    # get summary_tree first 
    if i==0:
        res = y - np.mean(y)
    else:
        res = y - staged_predict[i-1]
        
    summary_tree = summarize_tree(ensemble_tree[i, 0].tree_)
    explainer = shap.TreeExplainer(ensemble_tree[i, 0])
    loss += loss_treeshap(x, res, summary_tree, explainer, alpha)


#print("Loss sum + intercept: " + str(np.sum(loss, axis=1) + (y - np.mean(y)) ** 2))
#print("Real loss: " + str((y - ypred) ** 2) + "\n")

#explainer.shap_values(x)

#shap.TreeExplainer(tree_regressor).shap_values(x)


ypred = tree_regressor.predict(x)
sst = np.sum((y - np.mean(y)) ** 2)
sse = np.sum((y - ypred) ** 2)
rsq = 1 - sse/sst

sst = np.sum((y-np.mean(y)) ** 2)
rsq_res = 0 - np.sum(loss, axis=0) / sst
end = time.time()
print(end-start)

#print(loss)
print(np.round(rsq_res, 3))
print("treeshap_rsq: " + str(np.sum(rsq_res)))
print("Model rsq: "+ str(rsq) + "\n")

/var/folders/rn/qq25rs5s3wn9yqrktd830cvc0000gp/T/ipykernel_12542/2204973113.py:90: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  init_prediction[v] = tree.value[v] * n_v/n


8.408384084701538
[0.    0.    0.    0.    0.037 0.    0.076 0.    0.    0.    0.    0.
 0.103 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.038 0.049 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.313 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.02
 0.    0.    0.    0.    0.    0.306 0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.029 0.    0.    0.
 0.    0.    0.    0.   ]
treeshap_rsq: 0.974656574862846
Model rsq: 0.9746565748628457



# XGBOOST

## Formatter of xgboost since the data structure for xgboost is different

In [15]:
import xgboost
import uuid
import json
#print(xgb_regressor.get_booster().get_dump())
# We need 
# children_left: left children_index
# children_right: right children_index
# feature: array of features splitted at each node
# threshold: array of thresholds for corresponding splitting features
# max_depth: max_depth of the tree
# n_node_samples: array of sample size for each node
# value: array of values for each node, only leaf value is used, so only keep leaf value is fine
# node_count: total number of nodes 


x, y, coefficients = make_regression(n_samples=1000, n_features=100, n_informative=3, coef=True, random_state=0)

max_depth = 3

#xgb_regressor = xgb.XGBRegressor(n_estimators = 100, max_depth=max_depth, eta=0.1, alpha=0, reg_lambda=0)
xgb_regressor = xgboost.XGBRegressor(n_estimators = 100)
xgb_regressor.fit(x, y)

# dump
#model_dump = xgb_regressor.get_booster().get_dump()
#print(model_dump)
#raw = xgb_regressor.get_booster().save_raw()
#raw.read('f')

#xgb_regressor.save_model("xgb_model_tmp.json")
unique_filename = str(uuid.uuid4())
model_filename = f"xgb_model_{unique_filename}.json"
xgb_regressor.save_model(model_filename)

# import json
# with open('xgb_model_tmp.json', 'r') as file:
#     model_data = json.load(file)
    
#
# Load the model data
with open(model_filename, 'r') as file:
    model_data = json.load(file)

# Delete it after loading 
os.remove(model_filename)

def xgb_formatter(model_data, max_depth):
    """
    This function takes the json format of the xgboost output and transform it to a list that treeshap rsq can understand
    
    Parameters:
    model_data: json file
    max_depth: the max tree depth
    
    Examples:
    import json
    xgb_regressor.save_model("model.json")
    with open('model.json', 'r') as file:
    model_data = json.load(file)
    xgb_tree_res = xgb_formatter(model_data, 4)
    """
    
    trees_data = model_data["learner"]["gradient_booster"]["model"]["trees"]

    xgb_tree = []

    for tree in trees_data:
        xgb_tree.append(simple_tree(np.array(tree["left_children"]),
                                    np.array(tree["right_children"]), 
                                    np.array(tree["split_indices"]),
                                    np.array(tree["split_conditions"]),
                                    max_depth, 
                                    np.array(tree["sum_hessian"]), 
                                    np.array(tree["base_weights"]), 
                                    int(tree["tree_param"]["num_nodes"])))

    return(xgb_tree)
        

# be careful about the sum_hessian here, it may not be reliable other than squared loss: https://stackoverflow.com/questions/33520460/how-is-xgboost-cover-calculated
#print(trees_data)
# Dataframe
# xgb_regressor.get_booster().trees_to_dataframe()
start = time.time()
xgb_res = xgb_formatter(model_data, max_depth)
print(time.time()-start)

0.0015230178833007812


In [16]:
isinstance(xgb_regressor, xgboost.sklearn.XGBRegressor)
print(xgb_regressor.get_params().get("max_depth", 6))

None


## Testing, testing

In [17]:
import warnings
n = 5000
x, y, coefficients = make_regression(n_samples=n, n_features=100, n_informative=5, coef=True, random_state=0)

#sigma = np.random.normal(0, 2, size=n)
#y = y + sigma
max_depth = 1
learning_rate = 0.05
xgb_regressor = xgboost.XGBRegressor(n_estimators = 1, max_depth=max_depth, eta=learning_rate, alpha=0, reg_lambda=0)
#xgb_regressor = xgboost.XGBRegressor(eta=eta)
xgb_regressor.fit(x, y)
print("Finish")


xgb_booster = xgb_regressor.get_booster()
xgb_regressor.save_model("model.json")

import json
with open('model.json', 'r') as file:
    model_data = json.load(file)
    
xgb_res = xgb_formatter(model_data, max_depth)

base_score = np.float64(model_data["learner"]["learner_model_param"]['base_score'])

num_tree = len(xgb_res)

warnings.filterwarnings("ignore", module="xgb")

start = time.time()
loss = np.zeros_like(x)
for i in range(num_tree):
    # get summary_tree first 
    if i==0:
        #res = y - np.mean(y)
        res = y - base_score
    else:
        #  here is different
        res = y - xgb_regressor.predict(x, iteration_range=(0, i))
        
    # here is different
    summary_tree = summarize_tree(xgb_res[i])
    # differet
    explainer = shap.TreeExplainer(xgb_booster[i])
    
    # learning rate is different
    loss += loss_treeshap(x, res, summary_tree, explainer, 1)

#print("Loss sum + intercept: " + str(np.sum(loss, axis=1) + (y - np.mean(y)) ** 2))
#print("Real loss: " + str((y - ypred) ** 2) + "\n")

#explainer.shap_values(x)

#shap.TreeExplainer(tree_regressor).shap_values(x)

# first_pred = xgb_regressor.predict(x, iteration_range=(0, 1))
# second_pred = xgb_regressor.predict(x, iteration_range=(0, 2))
# third_pred = xgb_regressor.predict(x, iteration_range=(0, 3))

# final_pred =  xgb_regressor.predict(x)

#before_first = xgb_regressor.predict(x, iteration_range=(0, 1)) - xgb_booster[0].predict(xgboost.DMatrix(x))
#y_mean = np.mean(y)
#print(before_first - y_mean)
#print(before_first)


#print(np.corrcoef(y, first_pred))
# print(np.corrcoef(y, second_pred))
# print(np.corrcoef(y, third_pred))
# print(np.corrcoef(y, final_pred))

ypred = xgb_regressor.predict(x)
sst = np.sum((y - np.mean(y)) ** 2)
sse = np.sum((y - ypred) ** 2)
rsq = 1 - sse/sst

sst = np.sum((y-np.mean(y)) ** 2)
rsq_res = 0 - np.sum(loss, axis=0) / sst
end = time.time()
print(end-start)


print(rsq_res)
print("treeshap_rsq: " + str(np.sum(rsq_res)))
print("Model rsq: "+ str(rsq) + "\n")

Finish
0.31776905059814453
[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.03007301 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.    